In [1]:
import pandas as pd
import json
import csv
import sys
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import string
import numpy as np
from nltk.corpus import stopwords
import string
import emoji
import regex
import string
from datetime import timedelta
import matplotlib.dates as mdates
import seaborn as sn
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from unidecode import unidecode as unidec

In [2]:
dir_="C:\\Users\\Alex\\Personal-Code\\ACE_592_Dating_Sim\\"

In [3]:
#verification
import nltk
nltk.download('stopwords');
stopwords.words('english');
from nltk.corpus import stopwords;
stopwords.words('english');
print (stopwords.words('english'));

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#reads the csv files into memory for each dating app.
#Data has already been cleaed and all three files share the same columns
dfT = pd.read_csv(dir_ + "Wrangled_Data\\clean_Tinder.csv")
dfB = pd.read_csv(dir_ + "Wrangled_Data\\clean_Bumble.csv")
dfH = pd.read_csv(dir_ + "Wrangled_Data\\clean_Hinge.csv")

In [5]:
dfT.head(5);

In [6]:
#converts the time_stamp and date columns to datetime64 types
def to_datetime(df):
    df['date'] = pd.to_datetime(df.date)
    df['time_stamp'] = pd.to_datetime(df['time_stamp'])
    df_data = df.loc[df['score'] != 0]
    return df_data

#converts the clean text to UNI-8 for sentiment analysis
# def to_UNI8(df):
    

dfT = to_datetime(dfT)
dfB = to_datetime(dfB)
dfH = to_datetime(dfH)

dfT = dfT[['userName', 'full_text', 'clean_text', 'score', 'thumbs', 'date', 'time_stamp', 'reply_status', 'reply_content']]
dfB = dfB[['userName', 'full_text', 'clean_text', 'score', 'thumbs', 'date', 'time_stamp', 'reply_status', 'reply_content']]
dfH = dfH[['userName', 'full_text', 'clean_text', 'score', 'thumbs', 'date', 'time_stamp', 'reply_status', 'reply_content']]

In [7]:
#clips the df's into just the reviews from 2018-2022
df_T = dfT.loc[dfT['time_stamp'] >= np.datetime64('2018-01-01 00:00:00+00:00')]
df_B = dfB.loc[dfB['time_stamp'] >= np.datetime64('2018-01-01 00:00:00+00:00')]
df_H = dfH.loc[dfH['time_stamp'] >= np.datetime64('2018-01-01 00:00:00+00:00')]

C:\Users\Alex\AppData\Local\Temp\ipykernel_15200\2723410026.py:2: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  df_T = dfT.loc[dfT['time_stamp'] >= np.datetime64('2018-01-01 00:00:00+00:00')]
C:\Users\Alex\AppData\Local\Temp\ipykernel_15200\2723410026.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  df_B = dfB.loc[dfB['time_stamp'] >= np.datetime64('2018-01-01 00:00:00+00:00')]
C:\Users\Alex\AppData\Local\Temp\ipykernel_15200\2723410026.py:4: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  df_H = dfH.loc[dfH['time_stamp'] >= np.datetime64('2018-01-01 00:00:00+00:00')]


In [8]:
df_T.shape

(303647, 9)

### Let's take the whole dataframe (dropping reviews without comments) and analyze the sentiment of each review. Let's also examine the top 10 most positive and negative reviews.

In [9]:
df_T.head(10);

In [10]:
df_T.dtypes

userName                 object
full_text                object
clean_text               object
score                     int64
thumbs                    int64
date             datetime64[ns]
time_stamp       datetime64[ns]
reply_status               bool
reply_content            object
dtype: object

### Topic Analysis using unsupervised machine learning: Latent Dirichlet Allocation

### Sentiment Analysis using Vader

In [11]:
def get_sent(df):
    sid = SentimentIntensityAnalyzer()

    #takes the clean text and encodes it in utf-8, otherwise Vader will be most unpleased
    clean = df['clean_text']
    clean = str(clean).encode('utf-8')

    #calculates the sentiment of each review and stores the results in the 
    df1 = df.copy()
    df1['comp_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['compound'])
    df1['neg_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['neg'])
    df1['pos_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['pos'])
    df1['neu_sent'] = df['clean_text'].apply(lambda clean:sid.polarity_scores(str(clean))['neu'])
    
    
    return df1

In [12]:
#collects the sentiment scores for each of the trimmed dataframes
df_T = get_sent(df_T)
df_B = get_sent(df_B)
df_H = get_sent(df_H)

### Let's gather length of review as well and see if that will corrolate with review score
#### Then I should calc the correlation coeff for score with each of the sentiments and review length

In [13]:
def get_wc(df):
    #throws the copy error warning because it annoys me
    pd.options.mode.chained_assignment = None  # default='warn'
    
    #drops the columns without text. We won't be using them for the RF since they can't return sentiment scores
    df1 = df.dropna(subset = ['clean_text']).reset_index()
    #adds a WC column to the dataframe    
    df1['word_count'] = np.zeros(len(df1))
    
    
    #Crappy for loop to do this but it's whatever because we have no shortage of time
    #runs through the clean text in each row and counts the num of characters. Stores that value in a new df 
    #which will be merged with the existing df
    for i in range(len(df1)):
        wc = len(df1['clean_text'][i])
        df1['word_count'][i] = wc
    
    df2 = df1[['userName', 'full_text','clean_text', 'score', 'comp_sent', 'neg_sent', 'pos_sent', 'neu_sent', 'word_count']]
    
    return df2

In [14]:
df_T_wc = get_wc(df_T)
df_B_wc = get_wc(df_B)
df_H_wc = get_wc(df_H)

In [15]:
#saves the new CSV files as clean data
df_T_wc.to_csv(dir_ + "Wrangled_Data\\sent_Tinder.csv")
df_B_wc.to_csv(dir_ + "Wrangled_Data\\sent_Bumble.csv")
df_H_wc.to_csv(dir_ + "Wrangled_Data\\sent_Hinge.csv")